# Land Cover Classification
The original land use mentoined in the Landuse_Ma columns of the massavi is out dated and donot match with the ground realities. The land use and total number of parcels in the given dataset are 

  | Land Use      | No. of Parcels | Area (Kanal, Approx.) |
  |---------------|----------------|------------------------|
  | Agriculture   | 5,759          | 32,729                 |
  | Stream        | 302            | 822                    |
  | Other         | 229            | 1,047                  |
  | Road/Street   | 172            | 1,668                  |
  | Graveyard     | 29             | 202                    |
  | Built-Up      | 24             | 602                    |


Some of the anomalies in this dataset are as follow
- The type mentioned in Land use column as agricultue has been converted into builtup, not suitable for crop classification dataset.
- The type mentioned in Land use column as agricultue has now both builtup and agriculture, should be cleaned to make it suiutable for crop classification.
- Some of the on ground agriculture parcels has been recorded as other in Land use column maynot participate in crop classificaiton dataset as per this dataset.
## Purpose
The purpose of this exercise is to get clean agriculture parcels dataset for crop classificaiton using artifical intelligence algorithms like random forest, SVM etc. 

## Data Cleansing Methodology:
Since the agriculture column in the Land use column does not give us a clear idea of the current land type therefore, we have to use other means to computer the current land use type. Therefore, as first step we will filter the 5,759 agriculter parcels and compute NDVI and NDBI.
1. Get imagery from the ee for the time peroid. In this case i got 135 images for the year 2024. 
2. Pure agriculture: if NDVI is >= 3 and <= 7  the current_land_type is agriculture
3. Pure builtup: if the NDBI value is >0 and NDVI value is <=2 the current_land_type is builtup
4. Mix: compute the percentage parcel coverage of NDVI and NDBI and use a threshold to classify either builtup or agriculture
The resultant dataset will be clean revised agriculture parcels and can be used for crop classification.





In [158]:
#Step 1: Load the original dataset having Landuse_Ma columon. This dataset also have crop information. 

import geopandas as gpd
import pandas as pd
import numpy as np
import leafmap
import geemap
import ee

In [173]:
cropfile = 'pabbi_crop.geojson'
gdf = gpd.read_file(cropfile)
gdf['Landuse_Ma'].groupby(gdf['Landuse_Ma']).count().sort_values(ascending=False)

Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: Landuse_Ma, dtype: int64

In [160]:

selected_parcels = gdf[gdf['Landuse_Ma'] == 'Agriculture']
print(f'Total Agriculture Parcels are :{len(selected_parcels)}') 
# These are the parcels with Landuse_Ma as Agriculture and will be used for further analysis.

Total Agriculture Parcels are :5759


## Data Cleansing
1. Calculating NDVI and NDBI of the selected agriculture parcels : 5759.

In [161]:
# Lets Convert the gdf of the original file to ee features utilizing earth engine power
ee.Initialize()
ee.Authenticate()
boundary = geemap.gdf_to_ee(gdf) # this is for cliping the images
selectedparcels_fc = geemap.gdf_to_ee(selected_parcels) # this is for getting the agriculture parcels features Collection 


In [162]:
# Define a function to get imagery an apply filter
def get_collection (start_date,end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    filtered_collection = collection.filterDate(start_date, end_date).filterBounds(boundary.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    return filtered_collection

In [163]:
# defing ndvi , NDBI , BUI and UI indices

def get_ndvi(filtered_collection):
    # Calculate the annual NDVI from the filtered collection (filtered_collection)
    ndvi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
    # Calculate the mean NDVI for the year
    mean_ndvi = ndvi_collection.mean()
    return mean_ndvi
    
def get_ndbi(filtered_collection):
    # Calculate Normalized Difference Built-up Index (NDBI) (SWIR−NIR)/(SWIR+NIR) from the filtered collection filtered_collection
    # NDBI = (B11 - B8) / (B11 + B8)
    # Built‑up areas (positive values) vs. vegetation/water (negative)
    ndbi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B11', 'B8']).rename('NDBI'))
    # Calculate the mean NDBI for the year
    mean_ndbi = ndbi_collection.mean()
    return mean_ndbi
def get_ndwi(filtered_collection):
    # Calculate Normalized Difference Water Index (NDWI) (Green-NIR)/(Green+NIR) from the filtered collection filtered_collection
    # NDWI = (B3 - B8) / (B3 + B8)
    # Water (positive values) vs. built-up/vegetation (negative)
    ndwi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B3', 'B8']).rename('NDWI'))
    # Calculate the mean NDWI for the year
    mean_ndwi = ndwi_collection.mean()
    return mean_ndwi

def get_bui(filtered_collection):
    # Calculate the annual Built-Up Index (BUI) for an ee.ImageCollection `filtered_collection`.
    # BUI: (B11 + B4 - B8 - B2) / (B11 + B4 + B8 + B2)
    # Enhances urban density separation from other land‑covers
    bui_collection = filtered_collection.map(lambda img: (img.select('B11').add(img.select('B4')).subtract(img.select('B8'))
           .subtract(img.select('B2'))
           .divide(
               img.select('B11').add(img.select('B4'))
                  .add(img.select('B8'))
                  .add(img.select('B2'))
           )
           .rename('BUI')      
                    ))
    mean_bui = bui_collection.mean()
    return mean_bui
    
def get_ui(filtered_collection):
    # Calculate Urban Index (UI) (NDBI−NDVI)/(NDBI+NDVI) for the collection
    # Separates built‑up from vegetation more robustly
    mean_ndvi = get_ndvi(filtered_collection)
    mean_ndbi = get_ndbi(filtered_collection)
    mean_ui = mean_ndbi.subtract(mean_ndvi).divide(mean_ndbi.add(mean_ndvi)).rename('UI')
    return mean_ui



In [164]:
def get_composite(ndvi_img, ndbi_img, bui_img, ui_img,ndwi_img):
    """
    Stack the four annual‐mean index images into one composite,
    then clip it to the boundary.
    """
    # 1. Concatenate the four single‐band images
    composite_img = ee.Image.cat([ndvi_img, ndbi_img, bui_img, ui_img,ndwi_img])
    
    # 2. Rename the bands in the correct order
    composite_img = composite_img.rename(['NDVI','NDBI','BUI','UI','NDWI'])
    
    # 3. Clip to your boundary
    composite_img = composite_img.clip(boundary.geometry())
    
    return composite_img


In [165]:
filtered_collection = get_collection('2024-01-01', '2024-12-31')
ndvi_img = get_ndvi(filtered_collection)
ndbi_img = get_ndbi(filtered_collection)
bui_img = get_bui(filtered_collection)
ui_img = get_ui(filtered_collection)
ndwi_img = get_ndwi(filtered_collection)

composite_img = get_composite(ndvi_img, ndbi_img, bui_img, ui_img, ndwi_img)

In [166]:
filtered_collection.size().getInfo() # this will give the number of images in the collection

135

## Create a binary mask for each pixel in the composite.
1. if the NDVI value of a pixel is greater than 0.25 it is agricultue pixel
2. if the NDBI value of a pixel is greater than 0.0 than it is a builtup pixel
3. if the NDWI value of a pixel is greate than 0.3 than its water.

In [167]:
builtmask = composite_img.select('BUI').gt(0)
agrimask = composite_img.select('NDVI').gt(0.25)
watermask = composite_img.select('NDWI').gt(0.3)

In [174]:
  # Getting pixel count for calculating percentage of parcel and plx mean for pure agri/built 
def sum_count(composite_img, builtmask, agrimask,watermask ,parcels_fc):
    
    built = builtmask.rename('built_count').toInt()
    agri  = agrimask.rename('agri_count').toInt()
    water = watermask.rename('water_count').toInt()

    mask_img = ee.Image.cat([built, agri,water])

    pixel_sum = mask_img.reduceRegions(
        collection=parcels_fc,
        reducer=ee.Reducer.sum(),
        scale=10,
        crs=composite_img.projection()
    )
    return pixel_sum

def mean_indices(composite_img, parcels_fc):
    ndvi = composite_img.select('NDVI')
    ndbi = composite_img.select('NDBI')
    ui = composite_img.select('UI')
    bui = composite_img.select('BUI')
    ndwi = composite_img.select('NDWI')

    index_img = ee.Image.cat([ndvi,ndbi,bui,ui,ndwi])
    mean_img = index_img.reduceRegions(
        collection=parcels_fc,
        reducer=ee.Reducer.mean(),
        scale=10,
        crs=composite_img.projection()
    )
    return mean_img


In [175]:
pxlsum = sum_count(composite_img, builtmask, agrimask,watermask, selectedparcels_fc)
pxlmean = mean_indices(composite_img, selectedparcels_fc)
# Convert the ee.FeatureCollection to a pandas DataFrame
pxlsum_gdf= geemap.ee_to_gdf(pxlsum)
pxlmean_gdf= geemap.ee_to_gdf(pxlmean)  


In [182]:
# since both dataframes have duplicate columns therefore to avoid duplicaiton 
# we will select the columns we need for merging
mean_cols = ['geometry','NDVI','NDBI','BUI','UI','NDWI']
plxmean_small = pxlmean_gdf[mean_cols]

#Now we will merge the two dataframes
stats_gdf = pxlsum_gdf.merge(plxmean_small, on='geometry', how='left')

print(stats_gdf.columns.tolist())

['geometry', 'Area_Acre', 'Crop_Type', 'FFID', 'Landuse_Ma', 'Mouza_Name', 'Parcel_ID', 'agri_count', 'built_count', 'water_count', 'NDVI', 'NDBI', 'BUI', 'UI', 'NDWI']


In [183]:
# Now stats_gdf has columns 'built_count' and 'agri_count' per parcel
# Lets compute the percentage of built and agri land
stats_gdf['pct_built'] = stats_gdf['built_count'] / (stats_gdf['built_count'] + stats_gdf['agri_count']+ stats_gdf['water_count'])
stats_gdf['pct_agri']  = stats_gdf['agri_count']  / (stats_gdf['built_count'] + stats_gdf['agri_count']+ stats_gdf['water_count'])
stats_gdf['pct_water'] = stats_gdf['water_count']  / (stats_gdf['built_count'] + stats_gdf['agri_count']+stats_gdf['water_count'])
# Now we will drop the columns we do not need

In [ ]:
stats_gdf

['geometry',
 'Area_Acre',
 'Crop_Type',
 'FFID',
 'Landuse_Ma',
 'Mouza_Name',
 'Parcel_ID',
 'agri_count',
 'built_count',
 'water_count',
 'NDVI',
 'NDBI',
 'BUI',
 'UI',
 'NDWI',
 'pct_built',
 'pct_agri',
 'pct_water']

In [ ]:
def classify_hybrid(row,
                    ndvi_ag=0.6, ui_built=0.2,
                    maj_pct=0.75, min_pct=0.25):
    ndvi = row['NDVI']
    ui   = row['UI']
    pa   = row['pct_agri']
    pb   = row['pct_built']
    pw   = row['pct_water']
    ndwi = row['NDWI']

    # 1) Spectral “easy wins”
    if pw > maj_pct:
        return 'Water'
    if ndvi >= ndvi_ag and ui <= 0:
        return 'Pure-Agriculture'
    if ui >= ui_built:
        return 'Pure-Builtup'

    # 2) Percent-based majority
    if pa >= maj_pct:
        return 'Pure-Agriculture'
    if pb >= maj_pct:
        return 'Pure-Builtup'

    # 3) Partial by percent
    if pa >= min_pct and pa > pb:
        return 'Partial-Agriculture'
    if pb >= min_pct and pb > pa:
        return 'Partial-Builtup'

    # 4) Everything else → Mixed
    return 'Mixed'

# Apply and inspect
stats_gdf['landuse_class'] = stats_gdf.apply(classify_hybrid, axis=1)
print(stats_gdf['landuse_class'].value_counts())


landuse_class
Pure-Agriculture       2945
Partial-Agriculture    1269
Pure-Builtup            812
Partial-Builtup         641
Mixed                    92
Name: count, dtype: int64


In [ ]:
pure_agri = stats_gdf[stats_gdf['landuse_class'] == 'Pure-Agriculture']
pure_built = stats_gdf[stats_gdf['landuse_class'] == 'Pure-Builtup']
partial_agri = stats_gdf[stats_gdf['landuse_class'] == 'Partial-Agriculture']
partial_built = stats_gdf[stats_gdf['landuse_class'] == 'Partial-Builtup']
mixed = stats_gdf[stats_gdf['landuse_class'] == 'Mixed']


In [188]:
m = leafmap.Map(center=(37.5, 70), )
m.add_basemap('Google Satellite')


# 1. Create a minimal GeoDataFrame with only geometry + landuse_class
popup_gdf = stats_gdf[['geometry', 'landuse_class']].copy()


legend_dict = {
    'Pure Agriculture':           '#00AA00',  # dark green
    'Pure Builtup':               '#0000FF',  # blue
    'Partial Agri(25-75%)':       '#FFA500',  # orange
    'Partial Builtup(25-75%)':    '#800080',  # purple
    'Mixed':                      '#FFFF00',  # yellow
}


m.add_gdf(
    popup_gdf,
    layer_name='Landuse Class',
    style={                      # pick any style you like
        'color':   'black',
        'fillColor':'transparent',
        'fillOpacity': 0.1,
        'weight':  1
    },
    zoom_to_layer=True,
    info_mode='on_click'        # show popup when you click
)

# Pure Agriculture
m.add_gdf(
    pure_agri,
    layer_name='Pure Agriculture',
    style={
        'color': 'green',      # border
        'fillColor': 'green',  # interior
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
     
)

# Pure Builtup
m.add_gdf(
    pure_built,
    layer_name='Pure Builtup',
    style={
        'color': 'blue',
        'fillColor': 'blue',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Partial Agriculture
m.add_gdf(
    partial_agri,
    layer_name='Partial Agri (25-75%)',
    style={
        'color': 'orange',
        'fillColor': 'orange',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Partial Builtup
m.add_gdf(
    partial_built,
    layer_name='Partial Builtup(25-75%)',
    style={
        'color': 'purple',
        'fillColor': 'purple',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Other
m.add_gdf(
    mixed,
    layer_name='Other',
    style={
        'color': 'yellow',
        'fillColor': 'yellow',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Add legend
m.add_legend(
    legend_title='Landuse Classification',
    legend_dict=legend_dict,
    position='bottomleft'
)

m.add_layer_manager()
m




Map(center=[37.5, 70], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [187]:
stats_gdf.to_file('Pabbi_RevisedLandtype-V1.geojson', driver='GeoJSON')

In [137]:
m = leafmap.Map(center=(37.5, 70), zoom=6)
m.add_basemap('Google Satellite')

# 1. Create a minimal GeoDataFrame with only geometry + landuse_class
popup_gdf = stats_gdf[['geometry', 'landuse_class']].copy()

# 2. Add your styling layers *without* popups
layers_lsit = [ 
    (pure_agri,    'Pure Agriculture',    legend_dict['Pure Agriculture']),
    (pure_built,   'Pure Builtup',        legend_dict['Pure Builtup']),
    (partial_agri, 'Partial Agriculture', legend_dict['Partial Agriculture']),
    (partial_built,'Partial Builtup',     legend_dict['Partial Builtup']),
    (mixed,        'Mixed',               legend_dict['Mixed']),
        ]

for gdf, name, color in layers_lsit:
    # 2.1 Add the layer to the map   
    m.add_gdf(
        gdf,
        layer_name=name,
        style={
            'color':    color,
            'fillColor':color,
            'fillOpacity':0.3,
            'weight':   1
        },
        zoom_to_layer=True,
        info_mode='off'   # make sure no popups for these layers
    )

# 3. Add your minimal layer *last*, with popups ON only for landuse_class
m.add_gdf(
    popup_gdf,
    layer_name='Landuse Class',
    style={
        'color':      'black',
        'fillColor': 'transparent',
        'fillOpacity':0.1,
        'weight':     1
    },
    info_mode='on_click'  # only this layer shows a popup
)

# 4. Legend & Layer Manager
m.add_legend(
    legend_title='Landuse Classification',
    legend_dict=legend_dict,
    position='bottomleft'
)
m.add_layer_manager()

m


Map(center=[37.5, 70], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…